# Add trial labels

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/social-neuro-modeling'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [7]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.read_pickle("./proc/rce_pilot_2_04_lfp_spectral_sleap_spikes.pkl")

OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs



In [8]:
TRIAL_LABELS_DF = pd.read_excel("./rce_per_trial_labeling.xlsx")
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip().lower() for col in TRIAL_LABELS_DF.columns})
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])
TRIAL_LABELS_DF["time_stamp_index"] = TRIAL_LABELS_DF["time_stamp_index"].astype(np.int64)


In [9]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop_duplicates(subset=["session_dir", "time_stamp_index"])

In [10]:
TRIAL_LABELS_DF["video_file"].unique()

array(['20221202_134600_omission_and_competition_subject_6_1_and_6_2.1.videoTimeStamps.cameraHWSync',
       '20221203_154800_omission_and_competition_subject_6_4_and_6_1.1.videoTimeStamps.cameraHWSync',
       '20221214_125409_om_and_comp_6_1_and_6_3.1.videoTimeStamps.cameraHWSync',
       '20221215_145401_comp_amd_om_6_1_and_6_3.1.videoTimeStamps.cameraHWSync',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1.videoTimeStamps.cameraHWSync',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.videoTimeStamps.cameraHWSync',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1.videoTimeStamps.cameraHWSync',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230618

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [12]:
FULL_LFP_TRACES_PKL = "{}_lfp_traces_sleap_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [13]:
TRIAL_LABELS_DF.head()

,unnamed: 0,recording_file,video_file,time_stamp_index,video_frame,reward_frame,condition,competition_closeness,notes,experiment,time,subject_info,din,state,session_dir,video_number
84,2402.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,1734.0,1814.0,rewarded,NaN,NaN,NaN,6310663.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
85,2403.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,3728.0,3808.0,rewarded,NaN,NaN,NaN,7910662.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
86,2404.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,5972.0,6052.0,rewarded,NaN,NaN,NaN,9710660.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
87,2405.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,7966.0,8046.0,omission,NaN,NaN,NaN,11310658.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
88,2406.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,9836.0,9916.0,rewarded,NaN,NaN,NaN,12810657.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0


In [14]:
# grouped = TRIAL_LABELS_DF.groupby(['recording_file', 'video_file'])['time_stamp_index'].apply(lambda x: sorted(list(x))).reset_index()


grouped = TRIAL_LABELS_DF.groupby(['session_dir', 'video_file']).agg({'time_stamp_index': lambda x: list(x), 'condition': lambda x: list(x), 'competition_closeness': lambda x: list(x)}).reset_index()


In [15]:
grouped["video_file"] = grouped["video_file"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [16]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped, how='inner', left_on=['session_dir', 'video_name'], right_on=['session_dir', 'video_file'])

In [17]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = VIDEO_TO_FRAME_AND_SUBJECT_DF.rename(columns={'time_stamp_index': 'trial_time_stamps', 'condition': 'trial_label', 'competition_closeness': 'trial_competition_closeness'}, errors="ignore")

In [18]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[1.4, 1.1, 1.1, 1.4, 1.1, 1.4, 1.4, 1.4, 1.1, ...","[Subj 2 Only, Subj 1 Only, Subj 1 Only, Subj 2..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[1.4, 1.1, 1.1, 1.4, 1.1, 1.4, 1.4, 1.4, 1.1, ...","[Subj 2 Only, Subj 1 Only, Subj 1 Only, Subj 2..."


In [19]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"].apply(lambda x: np.array(x))
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].apply(lambda x: np.array(x))
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: np.array(x))

In [20]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].iloc[0]

array(['1.4', '1.1', '1.1', '1.4', '1.1', '1.4', '1.4', '1.4', '1.1',
       '1.1', '1.1', '1.1', '1.1', '1.1', '1.1', '1.1', '1.1', '1.1',
       '1.1', '1.4', 'rewarded', 'rewarded', 'rewarded', 'rewarded',
       'rewarded', 'rewarded', 'rewarded', 'omission', 'rewarded',
       'rewarded', 'rewarded', 'rewarded', 'rewarded', 'omission',
       'rewarded', 'rewarded', 'rewarded', 'omission', 'rewarded',
       'rewarded'], dtype='<U32')

- Converting the trial label to win or lose based on who won the trial

In [21]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
     

In [22]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(
    lambda x: ["win" if str(label).strip() == str(x["subject"]).strip() else "lose" if str(label).strip() in x["all_subjects"] else label for label in x["trial_label"]], 
    axis=1)

- Adding the competition closeness as a column

In [23]:
all_competition_closeness = np.unique(np.concatenate(VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"]))

In [24]:
# competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].iloc[0]}
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else ("comp" if "subj" in k.lower() else np.nan) for k in all_competition_closeness}

In [25]:
competition_closeness_map

{'Subj 1 Only': 'non_comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 2 Only': 'non_comp',
 'Subj 2 blocking Subj 1': 'comp',
 'Subj 2 then Subj 1': 'comp',
 'nan': nan}

In [26]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: [competition_closeness_map[label] for label in x])

In [27]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[lose, win, win, lose, win, lose, lose, lose, ...","[non_comp, non_comp, non_comp, non_comp, non_c..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[win, lose, lose, win, lose, win, win, win, lo...","[non_comp, non_comp, non_comp, non_comp, non_c..."


In [28]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[lose, win, win, lose, win, lose, lose, lose, ...","[non_comp, non_comp, non_comp, non_comp, non_c..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[win, lose, lose, win, lose, win, win, win, lo...","[non_comp, non_comp, non_comp, non_comp, non_c..."


In [30]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
     

In [29]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.to_pickle("./proc/rce_pilot_2_05_lfp_spectral_sleap_spikes_labels.pkl")

In [39]:
import os

/bin/bash: conda: command not found
